In [161]:
%load_ext autoreload
%autoreload 2

from utils import GraphTransformerDataset, GraphTransformerDataModule
from model.graph_transformer import GraphTransformer
import numpy as np
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
import torch


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
K = 100
L = 100
C = 10
y = np.random.rand(K, L).astype(np.float32)
x = np.random.rand(K, L, C).astype(np.float32)
mask = np.ones([K, L]).astype(np.float32)
eval_mask = np.where(np.random.rand(K, L) < 0.2, 1, 0).astype(np.float32) * mask
space_coords = np.random.rand(K, 2).astype(np.float32)
time_coords = np.arange(L).astype(np.float32)
space_sigma = 0.2
space_threshold = 0.1
space_partitions_num = 5
window_size = 20
stride = 10
val_ratio = 0.1

# Create dataset
dataset = GraphTransformerDataset(y, x, mask, eval_mask, space_coords, time_coords, space_sigma, space_threshold, space_partitions_num, window_size=window_size, stride=stride, val_ratio=val_ratio)





Computing METIS partitioning...
Done!


In [163]:
y_dim = 1
x_dim = C
hidden_dims = [64, 64]
output_dim = 1
ff_dim = 128
n_heads = 1
n_layers = 2
dropout = 0.0
model = GraphTransformer(y_dim, x_dim, hidden_dims, output_dim, ff_dim, n_heads, n_layers, dropout)


In [164]:
datamodule = GraphTransformerDataModule(dataset, batch_size=4)

In [165]:
trainer = Trainer(max_epochs=1)
trainer.fit(model, datamodule)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | y_enc          | MLP                | 4.3 K  | train
1 | x_enc          | MLP                | 4.9 K  | train
2 | pe             | PositionalEncoding | 0      | train
3 | encoder_layers | ModuleList         | 75.3 K | train
4 | readout        | MLP                | 8.4 K  | train
  | other params   | n/a                | 64     | n/a  
--------------------------------------------------------------
92.9 K    Trainable params
0         Non-trainable params
92.9 K    Total params
0.371     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/kehuiyao/Desktop/thesis/spatiotemporal_imputation/.conda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


/Users/kehuiyao/Desktop/thesis/spatiotemporal_imputation/.conda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/kehuiyao/Desktop/thesis/spatiotemporal_imputation/.conda/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 12/12 [00:14<00:00,  0.84it/s, v_num=47, val_loss=0.260, train_loss=0.287]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 12/12 [00:14<00:00,  0.84it/s, v_num=47, val_loss=0.260, train_loss=0.287]


In [166]:
trainer.test(model, datamodule)

/Users/kehuiyao/Desktop/thesis/spatiotemporal_imputation/.conda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 12/12 [00:06<00:00,  1.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.2533581852912903
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.2533581852912903}]